In [2]:
import requests
import csv
import time
import random
from bs4 import BeautifulSoup
from typing import List, Dict, Optional, Tuple, Callable
import os
from datetime import datetime
import logging
from urllib.parse import urljoin

# **Get ContentLinks From each Pages**

In [7]:
def scrape_voa_pages(
    base_url: str,
    start_page: int = 1,
    end_page: int = 100,
    output_file: str = "voa_khmer_articles.csv",
    article_selector: str = 'div.media-block a.img-wrap',
    page_param: str = "p",
    delay: float = 1.0,
    max_retries: int = 3,
    append_mode: bool = False,
    custom_headers: Optional[Dict] = None
) -> List[Dict[str, str]]:
    """
    Scrape VOA news articles from paginated pages.

    Args:
        base_url: The base URL without page parameters (e.g., "https://khmer.voanews.com/z/2288")
        start_page: Starting page number
        end_page: Ending page number
        output_file: Path to save CSV file
        article_selector: CSS selector for article links
        page_param: URL parameter name for page number (default: "p")
        delay: Delay between requests in seconds
        max_retries: Maximum number of retries for failed requests
        append_mode: If True, append to existing file; if False, create new file
        custom_headers: Custom HTTP headers (defaults to common User-Agent)

    Returns:
        List of dictionaries with 'headline' and 'link' keys
    """

    # Set default headers if not provided
    headers = custom_headers or {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    }

    all_articles = []
    processed_pages = 0
    failed_pages = []

    print(f"🚀 Starting scrape: {base_url}")
    print(f"   📊 Pages: {start_page} to {end_page}")
    print(f"   💾 Output: {output_file}")

    for page_num in range(start_page, end_page + 1):
        # Construct URL with page parameter
        if "?" in base_url:
            url = f"{base_url}&{page_param}={page_num}"
        else:
            url = f"{base_url}?{page_param}={page_num}"

        print(f"   📄 Processing Page {page_num}...", end=" ")

        # Retry logic for failed requests
        for attempt in range(max_retries):
            try:
                response = requests.get(url, headers=headers, timeout=10)

                if response.status_code != 200:
                    if attempt < max_retries - 1:
                        print(f"Status {response.status_code}, retrying...", end=" ")
                        time.sleep(delay * 2)  # Longer delay for retries
                        continue
                    else:
                        print(f"⚠️ Failed (Status {response.status_code})")
                        failed_pages.append(page_num)
                        break

                soup = BeautifulSoup(response.text, 'html.parser')
                articles = soup.select(article_selector)

                if not articles:
                    print("No articles found")
                    if page_num == start_page:  # First page has no articles
                        print("   ⚠️ Check your article_selector or base URL")
                    break

                page_articles = []
                for a_tag in articles:
                    link = a_tag.get('href')
                    title = a_tag.get('title') or a_tag.get_text(strip=True)

                    # Fix relative links
                    if link:
                        link = urljoin(base_url, link)

                    if link and title:
                        page_articles.append({
                            'headline': title.strip(),
                            'link': link,
                            'source_page': page_num,
                            'scraped_date': time.strftime("%Y-%m-%d")
                        })

                all_articles.extend(page_articles)
                processed_pages += 1
                print(f"✅ Found {len(page_articles)} articles")
                break  # Success, break retry loop

            except requests.exceptions.Timeout:
                print(f"Timeout, attempt {attempt + 1}/{max_retries}...", end=" ")
                if attempt < max_retries - 1:
                    time.sleep(delay * 3)
                else:
                    print("❌ Max retries reached")
                    failed_pages.append(page_num)
            except Exception as e:
                print(f"❌ Error: {str(e)[:50]}...")
                failed_pages.append(page_num)
                break

        # Delay between pages (except after last page)
        if page_num < end_page:
            time.sleep(delay)

    # Remove duplicates based on link
    unique_articles = []
    seen_links = set()

    for article in all_articles:
        if article['link'] not in seen_links:
            seen_links.add(article['link'])
            unique_articles.append(article)

    # Save to CSV
    save_mode = 'a' if append_mode else 'w'
    write_header = not (append_mode and file_exists(output_file))

    try:
        with open(output_file, save_mode, newline='', encoding='utf-8-sig') as f:
            fieldnames = ['headline', 'link', 'source_page', 'scraped_date']
            writer = csv.DictWriter(f, fieldnames=fieldnames)

            if write_header:
                writer.writeheader()
            writer.writerows(unique_articles)

        print(f"\n🎉 Scraping Complete!")
        print(f"   ✅ Processed pages: {processed_pages}")
        print(f"   ❌ Failed pages: {len(failed_pages)} {f'({failed_pages})' if failed_pages else ''}")
        print(f"   📄 Unique articles: {len(unique_articles)}")
        print(f"   💾 Saved to: {output_file}")

    except Exception as e:
        print(f"❌ Error saving CSV: {e}")

    return unique_articles

def file_exists(filepath: str) -> bool:
    """Check if a file exists."""
    import os
    return os.path.exists(filepath)


In [ ]:
# Example usage for different VOA sections
if __name__ == "__main__":
    # Example 1: Original science section
    science_articles = scrape_voa_pages(
        base_url="https://khmer.voanews.com/z/2276",
        start_page=0,
        end_page=100,
        output_file="../data/raw/voa_khmer_culture.csv",
        article_selector='div.media-block a.img-wrap',
        delay=1
    )

🚀 Starting scrape: https://khmer.voanews.com/z/2276
   📊 Pages: 0 to 100
   💾 Output: /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_culture.csv
   📄 Processing Page 0... ✅ Found 12 articles
   📄 Processing Page 1... ✅ Found 12 articles
   📄 Processing Page 2... ✅ Found 12 articles
   📄 Processing Page 3... ✅ Found 12 articles
   📄 Processing Page 4... ✅ Found 12 articles
   📄 Processing Page 5... ✅ Found 12 articles
   📄 Processing Page 6... ✅ Found 12 articles
   📄 Processing Page 7... ✅ Found 12 articles
   📄 Processing Page 8... ✅ Found 12 articles
   📄 Processing Page 9... ✅ Found 12 articles
   📄 Processing Page 10... ✅ Found 12 articles
   📄 Processing Page 11... ✅ Found 12 articles
   📄 Processing Page 12... ✅ Found 12 articles
   📄 Processing Page 13... ✅ Found 12 articles
   📄 Processing Page 14... ✅ Found 12 articles
   📄 Processing Page 15... ✅ Found 12 articles
   📄 Processing Page 16... ✅ Found 12 articles
   📄 Processing Page 17... ✅ Found 12 art

In [ ]:
environment_articles = scrape_voa_pages(
        base_url="https://khmer.voanews.com/z/2281",
        start_page=0,
        end_page=100,
        output_file="../data/raw/voa_khmer_environment.csv",
        article_selector='div.media-block a.img-wrap',
        delay=1
    )

🚀 Starting scrape: https://khmer.voanews.com/z/2281
   📊 Pages: 0 to 100
   💾 Output: /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_environment.csv
   📄 Processing Page 0... ✅ Found 12 articles
   📄 Processing Page 1... ✅ Found 12 articles
   📄 Processing Page 2... ✅ Found 12 articles
   📄 Processing Page 3... ✅ Found 12 articles
   📄 Processing Page 4... ✅ Found 12 articles
   📄 Processing Page 5... ✅ Found 12 articles
   📄 Processing Page 6... ✅ Found 12 articles
   📄 Processing Page 7... ✅ Found 12 articles
   📄 Processing Page 8... ✅ Found 12 articles
   📄 Processing Page 9... ✅ Found 12 articles
   📄 Processing Page 10... ✅ Found 12 articles
   📄 Processing Page 11... ✅ Found 12 articles
   📄 Processing Page 12... ✅ Found 12 articles
   📄 Processing Page 13... ✅ Found 12 articles
   📄 Processing Page 14... ✅ Found 12 articles
   📄 Processing Page 15... ✅ Found 12 articles
   📄 Processing Page 16... ✅ Found 12 articles
   📄 Processing Page 17... ✅ Found 12

In [ ]:
rights_articles = scrape_voa_pages(
        base_url="https://khmer.voanews.com/z/2967",
        start_page=0,
        end_page=100,
        output_file="../data/raw/voa_khmer_rights.csv",
        article_selector='div.media-block a.img-wrap',
        delay=1
    )

🚀 Starting scrape: https://khmer.voanews.com/z/2967
   📊 Pages: 0 to 100
   💾 Output: /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_rights.csv
   📄 Processing Page 0... ✅ Found 12 articles
   📄 Processing Page 1... ✅ Found 12 articles
   📄 Processing Page 2... ✅ Found 12 articles
   📄 Processing Page 3... ✅ Found 12 articles
   📄 Processing Page 4... ✅ Found 12 articles
   📄 Processing Page 5... ✅ Found 12 articles
   📄 Processing Page 6... ✅ Found 12 articles
   📄 Processing Page 7... ✅ Found 12 articles
   📄 Processing Page 8... ✅ Found 12 articles
   📄 Processing Page 9... ✅ Found 12 articles
   📄 Processing Page 10... ✅ Found 12 articles
   📄 Processing Page 11... ✅ Found 12 articles
   📄 Processing Page 12... ✅ Found 12 articles
   📄 Processing Page 13... ✅ Found 12 articles
   📄 Processing Page 14... ✅ Found 12 articles
   📄 Processing Page 15... ✅ Found 12 articles
   📄 Processing Page 16... ✅ Found 12 articles
   📄 Processing Page 17... ✅ Found 12 arti

In [ ]:
science_articles = scrape_voa_pages(
        base_url="https://khmer.voanews.com/z/2288",
        start_page=0,
        end_page=100,
        output_file="../data/raw/voa_khmer_science.csv",
        article_selector='div.media-block a.img-wrap',
        delay=1,
    )

🚀 Starting scrape: https://khmer.voanews.com/z/2288
   📊 Pages: 0 to 100
   💾 Output: /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_science.csv
   📄 Processing Page 0... ✅ Found 12 articles
   📄 Processing Page 1... ✅ Found 12 articles
   📄 Processing Page 2... ✅ Found 12 articles
   📄 Processing Page 3... ✅ Found 12 articles
   📄 Processing Page 4... ✅ Found 12 articles
   📄 Processing Page 5... ✅ Found 12 articles
   📄 Processing Page 6... ✅ Found 12 articles
   📄 Processing Page 7... ✅ Found 12 articles
   📄 Processing Page 8... ✅ Found 12 articles
   📄 Processing Page 9... ✅ Found 12 articles
   📄 Processing Page 10... ✅ Found 12 articles
   📄 Processing Page 11... ✅ Found 12 articles
   📄 Processing Page 12... ✅ Found 12 articles
   📄 Processing Page 13... ✅ Found 12 articles
   📄 Processing Page 14... ✅ Found 12 articles
   📄 Processing Page 15... ✅ Found 12 articles
   📄 Processing Page 16... ✅ Found 12 articles
   📄 Processing Page 17... ✅ Found 12 art

# **Download Contents from links**

In [24]:
class VOAArticleScraper:
    """A reusable class for scraping VOA article content from a list of URLs."""

    def __init__(
        self,
        input_file: str,
        output_file: str,
        headers: Optional[Dict] = None,
        content_selector: str = 'div.wsw',
        max_workers: int = 1,
        min_delay: float = 1.0,
        max_delay: float = 3.0,
        timeout: int = 10,
        max_retries: int = 2,
        log_file: Optional[str] = None,
        encoding: str = 'utf-8-sig'
    ):
        """
        Initialize the VOA Article Scraper.

        Args:
            input_file: Path to CSV file with 'headline' and 'link' columns
            output_file: Path to save the scraped content CSV
            headers: HTTP headers for requests (defaults to common User-Agent)
            content_selector: CSS selector for article content
            max_workers: Number of concurrent workers (1 for sequential)
            min_delay: Minimum delay between requests in seconds
            max_delay: Maximum delay between requests in seconds
            timeout: Request timeout in seconds
            max_retries: Maximum retries for failed requests
            log_file: Optional path to log file
            encoding: File encoding for input/output
        """
        self.input_file = input_file
        self.output_file = output_file
        self.content_selector = content_selector
        self.max_workers = max_workers
        self.min_delay = min_delay
        self.max_delay = max_delay
        self.timeout = timeout
        self.max_retries = max_retries
        self.encoding = encoding

        # Set default headers if not provided
        self.headers = headers or {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
            "Accept-Language": "en-US,en;q=0.9",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
            "Connection": "keep-alive"
        }

        # Setup logging
        self.setup_logging(log_file)

    def setup_logging(self, log_file: Optional[str]):
        """Configure logging."""
        self.logger = logging.getLogger('VOAArticleScraper')
        self.logger.setLevel(logging.INFO)

        # Clear existing handlers
        self.logger.handlers.clear()

        # Console handler
        console_handler = logging.StreamHandler()
        console_formatter = logging.Formatter('%(levelname)s: %(message)s')
        console_handler.setFormatter(console_formatter)
        self.logger.addHandler(console_handler)

        # File handler if specified
        if log_file:
            file_handler = logging.FileHandler(log_file, encoding='utf-8')
            file_formatter = logging.Formatter(
                '%(asctime)s - %(levelname)s - %(message)s',
                datefmt='%Y-%m-%d %H:%M:%S'
            )
            file_handler.setFormatter(file_formatter)
            self.logger.addHandler(file_handler)

    def read_articles_from_csv(self) -> List[Dict[str, str]]:
        """
        Read article links from input CSV file.

        Returns:
            List of dictionaries with 'headline' and 'link' keys

        Raises:
            FileNotFoundError: If input file doesn't exist
            ValueError: If CSV doesn't have required columns
        """
        if not os.path.exists(self.input_file):
            raise FileNotFoundError(f"Input file not found: {self.input_file}")

        articles = []
        try:
            with open(self.input_file, 'r', encoding=self.encoding) as f:
                reader = csv.DictReader(f)

                # Check required columns
                required_columns = {'headline', 'link'}
                if not required_columns.issubset(reader.fieldnames or []):
                    missing = required_columns - set(reader.fieldnames or [])
                    raise ValueError(f"CSV missing required columns: {missing}")

                for row in reader:
                    # Clean and validate
                    if row.get('link') and row.get('headline'):
                        articles.append({
                            'headline': row['headline'].strip(),
                            'link': row['link'].strip()
                        })

            self.logger.info(f"Read {len(articles)} articles from {self.input_file}")
            return articles

        except Exception as e:
            self.logger.error(f"Error reading CSV: {e}")
            raise

    def extract_content(
        self,
        url: str,
        custom_extractor: Optional[Callable[[BeautifulSoup], str]] = None
    ) -> Tuple[str, str]:
        """
        Extract content from a single article URL.

        Args:
            url: Article URL
            custom_extractor: Optional custom extraction function

        Returns:
            Tuple of (status, content)
        """
        for attempt in range(self.max_retries):
            try:
                response = requests.get(
                    url,
                    headers=self.headers,
                    timeout=self.timeout
                )

                if response.status_code != 200:
                    if attempt < self.max_retries - 1:
                        time.sleep(2)  # Wait before retry
                        continue
                    return "error", f"HTTP {response.status_code}"

                soup = BeautifulSoup(response.text, 'html.parser')

                # Use custom extractor if provided
                if custom_extractor:
                    content = custom_extractor(soup)
                else:
                    content = self.default_extractor(soup)

                if not content:
                    return "no_content", "No content found"

                return "success", content.strip()

            except requests.exceptions.Timeout:
                if attempt < self.max_retries - 1:
                    time.sleep(3)
                    continue
                return "timeout", "Request timeout"
            except Exception as e:
                if attempt < self.max_retries - 1:
                    time.sleep(2)
                    continue
                return "error", str(e)

        return "error", "Max retries exceeded"

    def default_extractor(self, soup: BeautifulSoup) -> str:
        """Default content extraction logic."""
        # Try primary selector
        content_div = soup.select_one(self.content_selector)
        if content_div:
            # Remove script and style elements
            for element in content_div.find_all(['script', 'style', 'iframe']):
                element.decompose()
            return content_div.get_text(separator=' ', strip=True)

        # Fallback selectors for VOA
        fallback_selectors = [
            'div.article-content',
            'article div.content',
            '.article-body',
            '#article-content',
            'div.article-text'
        ]

        for selector in fallback_selectors:
            element = soup.select_one(selector)
            if element:
                return element.get_text(separator=' ', strip=True)

        return ""

    def process_single_article(
        self,
        article: Dict[str, str],
        index: int,
        total: int,
        custom_extractor: Optional[Callable] = None
    ) -> Dict[str, str]:
        """Process a single article with logging and delay."""
        headline = article['headline'][:50] + "..." if len(article['headline']) > 50 else article['headline']

        self.logger.info(f"[{index}/{total}] Processing: {headline}")

        # Extract content
        status, content = self.extract_content(article['link'], custom_extractor)

        # Prepare result
        result = {
            'headline': article['headline'],
            'link': article['link'],
            'content': content,
            'status': status,
            'scraped_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'attempt_number': index # This index is now relative to the articles being processed
        }

        # Log result
        if status == "success":
            self.logger.info(f"    ✓ Success ({len(content)} chars)")
        else:
            self.logger.warning(f"    ✗ {status}: {content[:50]}...")

        # Random delay between requests
        if index < total: # Delay only if not the last article being processed
            delay = random.uniform(self.min_delay, self.max_delay)
            time.sleep(delay)

        return result

    def run(
        self,
        start_from: int = 0,
        max_articles: Optional[int] = None,
        custom_extractor: Optional[Callable] = None,
        append_mode: bool = False,
        resume_mode: bool = False
    ) -> List[Dict[str, str]]:
        """
        Main execution method.

        Args:
            start_from: Start from this index (0-based) from the input file
            max_articles: Maximum number of articles to process *after* start_from
            custom_extractor: Custom extraction function
            append_mode: Append to existing output file
            resume_mode: Skip already processed articles (based on output file)

        Returns:
            List of scraped articles with content
        """
        try:
            # Read input articles
            all_articles = self.read_articles_from_csv()

            if not all_articles:
                self.logger.warning("No articles to process")
                return []

            # Apply start_from offset
            articles_to_consider = all_articles[start_from:]

            # Apply max_articles limit
            if max_articles is not None:
                articles_to_consider = articles_to_consider[:max_articles]

            # Resume mode: skip already processed articles
            processed_links = set()
            if resume_mode and os.path.exists(self.output_file):
                try:
                    with open(self.output_file, 'r', encoding=self.encoding) as f:
                        reader = csv.DictReader(f)
                        for row in reader:
                            if row.get('link'):
                                processed_links.add(row['link'])
                    self.logger.info(f"Found {len(processed_links)} already processed articles in {self.output_file}")
                except Exception as e:
                    self.logger.warning(f"Could not read output file for resume: {e}")

            articles_to_process = []
            for article in articles_to_consider:
                if not resume_mode or article['link'] not in processed_links:
                    articles_to_process.append(article)

            total_to_process = len(articles_to_process)
            self.logger.info(f"🚀 Starting to process {total_to_process} articles")
            self.logger.info(f"   From: {self.input_file}")
            self.logger.info(f"   To: {self.output_file}")
            self.logger.info(f"   Starting from global index: {start_from}") # Log the original start_from

            # Open output file
            mode = 'a' if append_mode else 'w'
            write_header = not (append_mode and os.path.exists(self.output_file))

            with open(self.output_file, mode, newline='', encoding=self.encoding) as f_out:
                fieldnames = ['headline', 'link', 'content', 'status', 'scraped_date', 'attempt_number']
                writer = csv.DictWriter(f_out, fieldnames=fieldnames)

                if write_header:
                    writer.writeheader()

                # Process articles
                results = []
                for i, article in enumerate(articles_to_process, start=1):
                    result = self.process_single_article(
                        article=article,
                        index=start_from + i, # Global index for logging
                        total=start_from + total_to_process, # Total for logging
                        custom_extractor=custom_extractor
                    )

                    results.append(result)
                    writer.writerow(result)

                    # Flush periodically
                    if i % 10 == 0:
                        f_out.flush()

            # Summary
            success_count = sum(1 for r in results if r['status'] == 'success')
            self.logger.info(f"\n✅ Processing Complete!")
            self.logger.info(f"   Processed: {len(results)} articles")
            self.logger.info(f"   Success: {success_count}")
            self.logger.info(f"   Failed: {len(results) - success_count}")
            self.logger.info(f"   Saved to: {self.output_file}")

            return results

        except Exception as e:
            self.logger.error(f"❌ Fatal error: {e}")
            raise

# Helper function for simple usage
def scrape_voa_content(
    input_file: str,
    output_file: str,
    content_selector: str = 'div.wsw',
    start_from: int = 0,
    max_articles: Optional[int] = None,
    min_delay: float = 1.0,
    max_delay: float = 3.0,
    resume: bool = False,
    append_mode: bool = False,
    log_to_file: bool = False
) -> List[Dict[str, str]]:
    """
    Simplified function for scraping VOA article content.

    Example usage:
        articles = scrape_voa_content(
            input_file='links.csv',
            output_file='content.csv',
            content_selector='div.article-content',
            start_from=0,
            max_articles=100
        )
    """
    # Generate log file name if requested
    log_file = None
    if log_to_file:
        log_dir = os.path.dirname(output_file) or '.'
        log_file = os.path.join(log_dir, 'scraper.log')

    scraper = VOAArticleScraper(
        input_file=input_file,
        output_file=output_file,
        content_selector=content_selector,
        min_delay=min_delay,
        max_delay=max_delay
    )

    return scraper.run(
        start_from=start_from,
        max_articles=max_articles,
        resume_mode=resume,
        append_mode=append_mode

    )

# Custom extractor example
def custom_voa_extractor(soup: BeautifulSoup) -> str:
    """Example custom extractor for special VOA formats."""
    # Try multiple selectors
    selectors = ['div.wsw', 'div.article-content', 'article .body']

    for selector in selectors:
        element = soup.select_one(selector)
        if element:
            # Remove unwanted elements
            for tag in element.find_all(['script', 'style', 'nav', 'footer', 'aside']):
                tag.decompose()

            # Get text with paragraph separation
            paragraphs = [p.get_text(strip=True) for p in element.find_all('p')]
            return '\n\n'.join(paragraphs)

    return ""

In [ ]:
science_articles = scrape_voa_content(
    input_file='../data/raw/voa_khmer_science.csv',
    output_file='../data/raw/voa_khmer_science_final.csv',
    content_selector='div.wsw',
    start_from=0,
    max_articles=566,
)

INFO: Read 566 articles from /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_science.csv
INFO:VOAArticleScraper:Read 566 articles from /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_science.csv
INFO: 🚀 Starting to process 566 articles
INFO:VOAArticleScraper:🚀 Starting to process 566 articles
INFO:    From: /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_science.csv
INFO:VOAArticleScraper:   From: /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_science.csv
INFO:    To: /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_science_final.csv
INFO:VOAArticleScraper:   To: /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_science_final.csv
INFO:    Starting from global index: 0
INFO:VOAArticleScraper:   Starting from global index: 0
INFO: [1/566] Processing: តើ​បញ្ញាសប្បនិម្មិត​ AI អាច​កំណត់​រក​ជំងឺមហារីក​សុ...
INFO:VOAArticleScraper:[1/566] Processing: តើ

In [ ]:
education_articles = scrape_voa_content(
        input_file='../data/raw/voa_khmer_education.csv',
        output_file='../data/raw/voa_khmer_education_2_final.csv',
        content_selector='div.wsw',
        start_from=501,
        max_articles=250,
        resume=False
    )

INFO: Read 916 articles from /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_education.csv
INFO:VOAArticleScraper:Read 916 articles from /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_education.csv
INFO: 🚀 Starting to process 250 articles
INFO:VOAArticleScraper:🚀 Starting to process 250 articles
INFO:    From: /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_education.csv
INFO:VOAArticleScraper:   From: /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_education.csv
INFO:    To: /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_education_2_final.csv
INFO:VOAArticleScraper:   To: /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_education_2_final.csv
INFO:    Starting from global index: 501
INFO:VOAArticleScraper:   Starting from global index: 501
INFO: [502/751] Processing: Hello VOA៖ យុវជន​មាន​ផល​ប្រយោជន៍​យ៉ាង​ច្រើន​ក្នុង​...
INFO:VOAArticleScraper:

In [ ]:

articles = scrape_voa_content(
        input_file='../data/raw/voa_khmer_culture.csv',
        output_file='../data/raw/voa_khmer_culture_final.csv',
        content_selector='div.wsw',
        max_articles=600,
        resume=False
    )

INFO: Read 1203 articles from /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_culture.csv
INFO:VOAArticleScraper:Read 1203 articles from /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_culture.csv
INFO: 🚀 Starting to process 600 articles
INFO:VOAArticleScraper:🚀 Starting to process 600 articles
INFO:    From: /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_culture.csv
INFO:VOAArticleScraper:   From: /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_culture.csv
INFO:    To: /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_culture.csv_final.csv
INFO:VOAArticleScraper:   To: /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_culture.csv_final.csv
INFO:    Start index: 0
INFO:VOAArticleScraper:   Start index: 0
INFO: [1/600] Processing: កម្ពុជាគ្រោងពង្រីកសារមន្ទីរដើម្បីដាក់វត្ថុបុរាណ«ដួ...
INFO:VOAArticleScraper:[1/600] Processing: កម្ពុជាគ្រោងពង្រីកសារម

In [ ]:
environment_articles = scrape_voa_content(
        input_file='../data/raw/voa_khmer_environment.csv',
        output_file='../data/raw/voa_khmer_environment_final.csv',
        content_selector='div.wsw',
        max_articles=250,
        resume=False
    )

INFO: Read 1212 articles from /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_environment.csv
INFO:VOAArticleScraper:Read 1212 articles from /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_environment.csv
INFO: 🚀 Starting to process 250 articles
INFO:VOAArticleScraper:🚀 Starting to process 250 articles
INFO:    From: /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_environment.csv
INFO:VOAArticleScraper:   From: /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_environment.csv
INFO:    To: /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_environment.csv_final.csv
INFO:VOAArticleScraper:   To: /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_environment.csv_final.csv
INFO:    Start index: 0
INFO:VOAArticleScraper:   Start index: 0
INFO: [1/250] Processing: ក្រសួង​បរិស្ថាន​ស្នើ​ឲ្យ​ពលរដ្ឋ​ប្រុង​ប្រយ័ត្ន​ចំព...
INFO:VOAArticleScraper:[1/250] Processing

In [ ]:
rights_articles = scrape_voa_content(
        input_file='../data/raw/voa_khmer_rights.csv',
        output_file='../data/raw/voa_khmer_rights_final.csv',
        content_selector='div.wsw',
        max_articles=150,
        resume=False
    )

INFO: Read 1212 articles from /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_rights.csv
INFO:VOAArticleScraper:Read 1212 articles from /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_rights.csv
INFO: 🚀 Starting to process 150 articles
INFO:VOAArticleScraper:🚀 Starting to process 150 articles
INFO:    From: /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_rights.csv
INFO:VOAArticleScraper:   From: /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_rights.csv
INFO:    To: /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_rights.csv_final.csv
INFO:VOAArticleScraper:   To: /content/drive/MyDrive/ITC/WR/KhmerTextClassification/dataset/voa_khmer_rights.csv_final.csv
INFO:    Start index: 0
INFO:VOAArticleScraper:   Start index: 0
INFO: [1/150] Processing: លោក រ៉ុង ឈុន ស្នើ​សុំ​បង់​ប្រាក់​សំណង​ប្រមាណ​១០​ម៉...
INFO:VOAArticleScraper:[1/150] Processing: លោក រ៉ុង ឈុន ស្នើ​សុំ​បង់​ប្